# Biostat 257 Homework 4 - Solutions

John Baierl

**Due May 20 @ 11:59PM**

In [3]:
versioninfo()

Julia Version 1.7.2
Commit bf53498635 (2022-02-06 15:21 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin21.2.0)
  CPU: Apple M1
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, cyclone)


We are going to try different numerical methods learnt in class on the [Google PageRank problem](https://en.wikipedia.org/wiki/PageRank).

## Q1 (5 pts) Recognize structure

Let $\mathbf{A} \in \{0,1\}^{n \times n}$ be the connectivity matrix of $n$ web pages with entries
$$
\begin{eqnarray*}
	a_{ij}= \begin{cases}
	1 & \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}.
\end{eqnarray*}
$$
$r_i = \sum_j a_{ij}$ is the out-degree of page $i$. That is $r_i$ is the number of links on page $i$. Imagine a random surfer exploring the space of $n$ pages according to the following rules.  

- From a page $i$ with $r_i>0$
    * with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page  
    * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
- From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
    
The process defines a Markov chain on the space of $n$ pages. Write the transition matrix $\mathbf{P}$ of the Markov chain as a sparse matrix plus rank 1 matrix.

#### Solution:

The probability for any paricular element of the transition matrix $\mathbf{P}$ is given by:

$p_{ij} = 
\begin{cases} 
      p \frac{a_{ij}}{r_i} + (1 - p) \frac{1}{n} & r_i > 0 \\
      \frac{1}{n} & r_i = 0
\end{cases}
$

Converting this to matrix notation, we can express the full transition matrix $\mathbf{P}$ as:

$\mathbf{P} = \textrm{diag}(\mathbf{d})\mathbf{A} + \mathbf{b} \mathbf{1}^T_n$

where $\mathbf{b} \in \mathbb{R}^{n}$, $b_i = 
\begin{cases} 
      (1 - p) \frac{1}{n} & r_i > 0 \\
      \frac{1}{n} & r_i = 0
\end{cases}$

and $\mathbf{d} \in \mathbb{R}^n$, $d_i = 
\begin{cases} 
      \frac{p}{r_i} & r_i > 0 \\
      0 & r_i = 0
\end{cases}$

The sparsity pattern of the first term matches the sparsity of $\mathbf{A}$, while the second term is a rank one matrix.

## Q2 Relate to numerical linear algebra

According to standard Markov chain theory, the (random) position of the surfer converges to the stationary distribution $\mathbf{x} = (x_1,\ldots,x_n)^T$ of the Markov chain. $x_i$ has the natural interpretation of the proportion of times the surfer visits page $i$ in the long run. Therefore $\mathbf{x}$ serves as page ranks: a higher $x_i$ means page $i$ is more visited. It is well-known that $\mathbf{x}$ is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix $\mathbf{P}$. That is $\mathbf{P}^T \mathbf{x} = \mathbf{x}$. Therefore $\mathbf{x}$ can be solved as an **eigen-problem**. It can also be cast as **solving a linear system**. Since the row sums of $\mathbf{P}$ are 1, $\mathbf{P}$ is rank deficient. We can replace the first equation by the $\sum_{i=1}^n x_i = 1$.

Hint: For iterative solvers, we don't need to replace the 1st equation. We can use the matrix $\mathbf{I} - \mathbf{P}^T$ directly if we start with a vector with all positive entries.

## Q3 (10 pts) Explore data

Obtain the connectivity matrix `A` from the `SNAP/web-Google` data in the MatrixDepot package. 

In [4]:
using MatrixDepot

md = mdopen("SNAP/web-Google")
# display documentation for the SNAP/web-Google data
mdinfo(md)

┌ Info: verify download of index files...
└ @ MatrixDepot /Users/johnbaierl/.julia/packages/MatrixDepot/GEDc3/src/MatrixDepot.jl:139
┌ Info: reading database
└ @ MatrixDepot /Users/johnbaierl/.julia/packages/MatrixDepot/GEDc3/src/download.jl:23
┌ Info: adding metadata...
└ @ MatrixDepot /Users/johnbaierl/.julia/packages/MatrixDepot/GEDc3/src/download.jl:67
┌ Info: adding svd data...
└ @ MatrixDepot /Users/johnbaierl/.julia/packages/MatrixDepot/GEDc3/src/download.jl:69
┌ Info: writing database
└ @ MatrixDepot /Users/johnbaierl/.julia/packages/MatrixDepot/GEDc3/src/download.jl:74
┌ Info: used remote sites are sparse.tamu.edu with MAT index and math.nist.gov with HTML index
└ @ MatrixDepot /Users/johnbaierl/.julia/packages/MatrixDepot/GEDc3/src/MatrixDepot.jl:141


# SNAP/web-Google

###### MatrixMarket matrix coordinate pattern general

---

  * UF Sparse Matrix Collection, Tim Davis
  * http://www.cise.ufl.edu/research/sparse/matrices/SNAP/web-Google
  * name: SNAP/web-Google
  * [Web graph from Google]
  * id: 2301
  * date: 2002
  * author: Google
  * ed: J. Leskovec
  * fields: name title A id date author ed kind notes
  * kind: directed graph

---

  * notes:
  * Networks from SNAP (Stanford Network Analysis Platform) Network Data Sets,
  * Jure Leskovec http://snap.stanford.edu/data/index.html
  * email jure at cs.stanford.edu
  * 
  * Google web graph
  * 
  * Dataset information
  * 
  * Nodes represent web pages and directed edges represent hyperlinks between them.
  * The data was released in 2002 by Google as a part of Google Programming
  * Contest.
  * 
  * Dataset statistics
  * Nodes   875713
  * Edges   5105039
  * Nodes in largest WCC    855802 (0.977)
  * Edges in largest WCC    5066842 (0.993)
  * Nodes in largest SCC    434818 (0.497)
  * Edges in largest SCC    3419124 (0.670)
  * Average clustering coefficient  0.6047
  * Number of triangles     13391903
  * Fraction of closed triangles    0.05523
  * Diameter (longest shortest path)    22
  * 90-percentile effective diameter    8.1
  * 
  * Source (citation)
  * 
  * J. Leskovec, K. Lang, A. Dasgupta, M. Mahoney. Community Structure in Large
  * Networks: Natural Cluster Sizes and the Absence of Large Well-Defined Clusters.
  * arXiv.org:0810.1355, 2008.
  * 
  * Google programming contest, 2002
  * http://www.google.com/programming-contest/
  * 
  * Files
  * File    Description
  * web-Google.txt.gz   Webgraph from the Google programming contest, 2002

---

916428 916428 5105039


In [5]:
# connectivity matrix
A = md.A

916428×916428 SparseArrays.SparseMatrixCSC{Bool, Int64} with 5105039 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿

Compute summary statistics:
* How much memory does `A` take? If converted to a `Matrix{Float64}` (don't do it!), how much memory will it take?  
* number of web pages
* number of edges (web links). 
* number of dangling nodes (pages with no out links)
* histogram of in-degrees  
* list the top 20 pages with the largest in-degrees?  
* histogram of out-degrees
* which the top 20 pages with the largest out-degrees?
* visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`. 

**Hint**: For plots, you can use the [UnicodePlots.jl](https://github.com/Evizero/UnicodePlots.jl) package (`spy`, `histogram`, etc), which is fast for large data. 

#### Solution:

**a)** How much memory does `A` take? If converted to a `Matrix{Float64}` (don't do it!), how much memory will it take?

In [6]:
Base.summarysize(A) #size of A in bytes

53276943

The sparse array A takes up 53,276,943 bytes, or approximately 53.3 MB of memory.

A reasonable approximation of the size of $A$ if converted to a dense matrix can be obtained by scaling this value based on the sparsity of $A$.

In [7]:
sparsity = 5105039 / 916428^2;

53276943 / sparsity

8.764697523993461e12

We should expect a dense version of $A$ to take about 8.76 TB of storage!

**b)** Number of web pages

The number of webpages in this simulated internet is equivalent to the number of rows (or columns) the square matrix $A$.  So there are 916,428 total web pages.

**c)** Number of edges (web links)

There are 5,105,039 nonzero elements of $\mathbf{A}$.  This corresponds with the total number of links in this miniature internet.

**d)** Number of dangling nodes (pages with no out links)

This is equivalent to the number of rows that have no non-zero elements.

In [8]:
using StatsBase

rowsums = sum(A, dims = 2);
c = counts(rowsums)[1]

176974

176,974 pages have no out links.

**e)** Histogram of in-degrees

In [9]:
using UnicodePlots

colsums = sum(A, dims = 1);
histogram(colsums, xscale =:log10, nbins = 52) #log scale for better visibility

                    ┌                                        ┐ 
   [   0.0,  200.0) ┤████████████████████████████████  915169  
   [ 200.0,  400.0) ┤███████████████▋ 812                      
   [ 400.0,  600.0) ┤████████████▍ 203                         
   [ 600.0,  800.0) ┤██████████▎ 80                            
   [ 800.0, 1000.0) ┤███████▉ 30                               
   [1000.0, 1200.0) ┤██████▊ 19                                
   [1200.0, 1400.0) ┤█████▊ 12                                 
   [1400.0, 1600.0) ┤████▎ 6                                   
   [1600.0, 1800.0) ┤████▎ 6                                   
   [1800.0, 2000.0) ┤█████▎ 9                                  
   [2000.0, 2200.0) ┤████▊ 8                                   
   [2200.0, 2400.0) ┤███▎ 4                                    
   [2400.0, 2600.0) ┤████▌ 7                                   
   [2600.0, 2800.0) ┤█████▋ 11                                 
   [2800.0, 3000.0) ┤████▎ 6            

This plot shows that while the vast majority of sites have relatively few in-degrees (< 200), a small number of sites weild dramatically more influecnce over the flow of internet traffic, with as many as 27x more in-degrees as the smallest bin.  This motivates the Markov chain PageRank approch as opposed to simply counting links.

**f)** List the top 20 pages with the largest in-degrees? 

In [10]:
colsums = vec(colsums);
b = sortperm(colsums);

rank_indeg = collect(zip(b, colsums[b]));
rank_indeg[(length(rank_indeg) - 19):length(rank_indeg)]

20-element Vector{Tuple{Int64, Int64}}:
 (687326, 3956)
 (173977, 3988)
 (384667, 4010)
 (765335, 4015)
 (213433, 4084)
 (407611, 4180)
 (459075, 4187)
 (558792, 4206)
 (41910, 4219)
 (551830, 4220)
 (828964, 4484)
 (605857, 4550)
 (819224, 4620)
 (163076, 4731)
 (885606, 4847)
 (32164, 5097)
 (751385, 5182)
 (504141, 5271)
 (597622, 5354)
 (537040, 6326)

The first value in each pair above gives the webpage number, and the second gives the number of in-degrees for that page.

**g)** Histogram of out-degrees

In [11]:
histogram(rowsums, xscale =:log10, nbins = 52) #log scale for better visibility

                  ┌                                        ┐ 
   [  0.0,  10.0) ┤████████████████████████████████  715808  
   [ 10.0,  20.0) ┤████████████████████████████▋ 175990      
   [ 20.0,  30.0) ┤███████████████████████▌ 20314            
   [ 30.0,  40.0) ┤██████████████████▍ 2314                  
   [ 40.0,  50.0) ┤████████████████▎ 908                     
   [ 50.0,  60.0) ┤██████████████▍ 421                       
   [ 60.0,  70.0) ┤████████████▊ 219                         
   [ 70.0,  80.0) ┤███████████▉ 152                          
   [ 80.0,  90.0) ┤██████████▎ 71                            
   [ 90.0, 100.0) ┤█████████▌ 53                             
   [100.0, 110.0) ┤█████████▎ 48                             
   [110.0, 120.0) ┤████████▋ 38                              
   [120.0, 130.0) ┤██████▋ 17                                
   [130.0, 140.0) ┤█████▊ 12                                 
   [140.0, 150.0) ┤█████▊ 12                                 
   [150.

Interestingly, the out-degree distribution shows less variability overall with a less extreme rightward (or in this above plot, downward) skew.  So, there is less variation in the number of out-degree links than the number of in-degree links in this miniature internet.

**h)** Which the top 20 pages with the largest out-degrees?

In [12]:
rowsums = vec(rowsums);
c = sortperm(rowsums);

rank_outdeg = collect(zip(c, rowsums[c]));
rank_outdeg[(length(rank_outdeg) - 19):length(rank_outdeg)]

20-element Vector{Tuple{Int64, Int64}}:
 (915274, 213)
 (302734, 216)
 (838279, 225)
 (321092, 229)
 (411594, 231)
 (579315, 244)
 (598189, 245)
 (282141, 247)
 (685696, 248)
 (667585, 253)
 (885729, 256)
 (156951, 257)
 (376429, 258)
 (600480, 265)
 (412411, 268)
 (808644, 277)
 (768092, 330)
 (203749, 372)
 (305230, 372)
 (506743, 456)

Once again, the first value in each pair gives the webpage number, and the second gives the total number of out-degrees for that page.

**i)** Visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`.

In [13]:
@views Asub = A[1:10000, 1:10000];

spy(Asub)

         ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀Sparsity Pattern⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀    
         ┌──────────────────────────────────────────┐    
       1 │⠀⠀⠀⠀⠀⠀⠀⠂⠀⠀⠀⠀⢀⢂⡂⠐⠀⠀⠀⠠⠀⠀⠀⠔⠄⠀⠈⢀⠀⠀⠉⠀⠀⠁⠀⠀⠐⠀⠄⠀⠐⠀│ > 0
         │⠀⠀⠀⠀⠀⠂⡂⠀⠄⠄⠈⠈⡔⠀⠀⠁⠂⠀⠀⠀⠀⠠⠁⠒⠁⠀⠄⠀⠐⠀⠈⠀⠀⡐⠀⠀⠠⠀⡀⠀⠉⠀│ < 0
         │⠀⠐⠠⠀⠀⠁⢐⠄⠀⠢⠠⠀⠀⡀⠈⠀⠋⠀⠀⠀⠀⠀⠂⡀⠀⠈⠄⠀⠄⠁⠈⠀⠠⠀⠰⠀⠀⠀⠀⢀⠆⠀│    
         │⠀⠈⠠⠂⠀⠈⠀⡠⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠈⠄⠁⢀⡀⠐⠄⠄⠀⠀⢀⠀⠈⠂⢠⠀⠢⠐⠀⠀⠀⡀│    
         │⠀⠀⠐⡁⠀⡀⠀⠀⠠⠀⠁⠀⠀⠀⡄⠀⠈⠁⠀⠀⠀⠀⠠⠂⠀⠀⠀⠀⠀⠂⠪⠀⠀⠂⠀⠀⠀⠄⢀⠀⠁⠀│    
         │⠂⢀⠀⠁⠀⠀⡀⡉⠀⠀⠠⠂⠐⢀⠄⠁⠐⠀⠈⠀⠀⠀⠀⠀⡀⠐⠁⡠⠀⠂⠚⠀⠁⠀⠐⢀⠆⠀⠈⠄⢀⠂│    
         │⠀⠀⠀⠁⠀⡄⠀⠀⠀⠀⠒⠀⢄⠄⢂⠀⠂⠀⠀⠄⠀⢀⠀⠣⠀⢀⠀⠀⠀⠀⠠⡀⠀⠀⢠⠄⠀⡀⠀⠂⠂⠀│    
         │⢀⠈⠀⠀⠂⠀⠀⠁⢠⠈⠄⠁⠀⠀⡈⠈⠠⠀⢀⡦⠀⠀⠅⠐⠀⠨⠁⠨⠀⠄⠠⢀⠀⠐⢐⠀⠀⠉⠀⠀⠠⠉│    
         │⠀⠀⠨⠀⠋⠀⠄⠀⠂⠀⠈⠀⠀⠀⠈⠀⠀⠀⠀⡑⠀⢀⠈⠈⠀⠀⠀⠈⠀⠂⠐⠼⠀⠠⠀⠀⡈⠁⠀⠀⠡⠄│    
         │⠠⡁⠀⠀⠀⠀⠀⠀⠀⠀⠂⢀⠀⠀⠀⡄⠀⠀⠀⠤⠀⠀⠁⠀⡂⠀⠀⠀⠄⠀⢈⠀⢔⡊⣨⠈⠂⠀⠄⢀⠀⠀│    
         │⠀⠂⠀⠈⠀⠀⠀⠀⠀⠀⠈⠀⡀⠀⠂⠠⠐⠀⠄⠀⠀⠀⠀⠀⠀⠤⠀⠀⠀⠠⠘⠐⠀⠄⢘⠀⠀⢐⠄⠀⠁⠀│    
         │⢀⠄⢠⠄⡊⠂⠢⢀⠀⠀⠀⠀⠄⡀⡑⠀⡐⠀⠁⡂⠀⠀⠀⠰⠂⢠⠄⠈⠀⠈⠐⠀⠀⠀⠉⠀⠀⡀⠀⠀⠃⠀│    
         │⠀⠀⠅⠀⡁⠀⠀⠀⠀⠀⢁⢀⠀⡀⡀⡐⠀⠀⠈⠈⠔⠄⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠄⢉⠠⠁⠀⠁⠀⠀⠀│    
         │⠊⢈⠀⠁⠀⠀⠀⠅⠀⠂⠀⡀⠀⠀⠀⠀⡀⠀⠀⠐⠄⠁⠠⠀⠐⠈⠀⠀⠀⡀⢠⠀⠀⠀⢰⠀⠀⠀⢀⠀⠠⠂│    
         │⠈⠀⠀⠈⠄⠀⠀⠄⠐⠀⠠⠁⠁⠀⡂⠀⠠⠄⠌⠅⣀⠀⠀⢀⠀⠠⠀⠠⠠⠀⠲⢂⠀⠰⡄⠀⢀⠬⠈⠀⠀⠀│    
         │⠈⠀⠀⠄

## Q4 (5 pts) Dense linear algebra? 

Consider the following methods to obtain the page ranks of the `SNAP/web-Google` data. 

1. A dense linear system solver such as LU decomposition.  
2. A dense eigen-solver for asymmetric matrix.  

For the LU approach, estimate (1) the memory usage and (2) how long it will take assuming that the LAPACK functions can achieve the theoretical throughput of your computer. 

#### Solution:

1. An LU decomposition for a dense matrix costs approximately $\frac{2}{3}n^3$ flops.  The forward and backwards substitutions necessary to then solve the linear system cost $n^2 + n^2 = 2n^2$ flops.  So in total, LU decomposition to optain page ranks costs $\frac{2}{3}n^3 + 2n^2$ flops.  The total flops for our 916,428 by 916,428 matrix, $\mathbf{A}$ are:

In [14]:
flops = (2 / 3) * 916428^3 + 2 * 916428^2 #total flop count

5.131037779285815e17

2. According to documentation, my computer (2021 MacBook Air) is capable of 2.6 teraflops per second of throughput.  Assuming this is fully achieved, we should expect this dense linear system solver to take:

In [15]:
flops / 2.6e12 / 60 / 60 / 24 #converting from seconds to days

2.2841158205510217

An LU decomposition to solve this dense linear system should take at least 2.28 days.

## Q5 (75 pts) Iterative solvers

Set the _teleportation_ parameter at $p = 0.85$. Consider the following methods for solving the PageRank problem. 

1. An iterative linear system solver such as GMRES. 
2. An iterative eigen-solver such as Arnoldi method.

For iterative methods, we have many choices in Julia. See a list of existing Julia packages for linear solvers at this [page](https://jutho.github.io/KrylovKit.jl/stable/#Package-features-and-alternatives-1). The start-up code below uses the [KrylovKit.jl](https://github.com/Jutho/KrylovKit.jl) package. You can use other packages if you prefer. Make sure to utilize the special structure of $\mathbf{P}$ (sparse + rank 1) to speed up the matrix-vector multiplication. 

### Step 1 (15 pts)

Let's implement a type `PageRankImPt` that mimics the matrix $\mathbf{M} = \mathbf{I} - \mathbf{P}^T$. For iterative methods, all we need to provide are methods for evaluating $\mathbf{M} \mathbf{v}$ and $\mathbf{M}^T \mathbf{v}$ for arbitrary vector $\mathbf{v}$.

#### Solution:

In [49]:
using BenchmarkTools, LinearAlgebra, SparseArrays, Revise

# a type for the matrix M = I - P^T in PageRank problem, key: don't actually form dg!

struct PageRankImPt{TA <: Number, IA <: Integer, T <: AbstractFloat} <: AbstractMatrix{T}
    A         :: SparseMatrixCSC{TA, IA} # adjacency matrix, data
    telep     :: T # parameter in pagerank algo
    # working arrays
    rowsum        :: Vector{Int64}
    storage_n     :: Vector{T}
    storage_n2    :: Vector{T}
    b             :: Vector{T}
    d             :: Vector{T}
end

# constructor
function PageRankImPt(A::SparseMatrixCSC, telep::T) where T <: AbstractFloat
    n = size(A, 1)

    rowsum        = vec(sum(A, dims = 2))
    storage_n     = Vector{T}(undef, n)
    storage_n2    = Vector{T}(undef, n)
    b             = Vector{T}(undef, n)
    d             = Vector{T}(undef, n)
    #Construct d with piecewise definition
    for i in 1:n
        if rowsum[i] == 0
            d[i] = 0
        else
            d[i] = telep / rowsum[i]
        end
    end
    #Construct vector b with piecewise definition
    for i in 1:n
        if rowsum[i] == 0
            b[i] = 1 / n
        else
            b[i] = (1 - telep) / n
        end
    end
    PageRankImPt(A, telep, rowsum, storage_n, storage_n2, b, d)
end

LinearAlgebra.issymmetric(::PageRankImPt) = false
Base.size(M::PageRankImPt) = size(M.A)
Base.getindex(M::PageRankImPt, i, j) = (i == j) - M.d[j] * M.A[j, i] - M.b[j]

# overwrite `out` by `(I - Pt) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        M   :: PageRankImPt{<:Number, <:Integer, T}, 
        v   :: Vector{T} # iterative solver keeps feeding v
        ) where T <: AbstractFloat    
    #Calculate A'diag(d)'v, store in storage_n2
    M.storage_n2 .= M.d .* v
    mul!(M.storage_n, transpose(M.A), M.storage_n2)
    #Piece it all together
    out .= v .- M.storage_n .- dot(M.b, v)
    return out
end

# overwrite `out` by `(I - P) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        Mt  :: Transpose{T, PageRankImPt{TA, IA, T}}, 
        v   :: Vector{T}
        ) where {TA<:Number, IA<:Integer, T <: AbstractFloat}
    M = Mt.parent
    #Calculate diag(d)Av, store in storage_n
    mul!(M.storage_n, M.A, v)
    #Piece it all together
    out .= v .- M.d .* M.storage_n .- sum(v) .* M.b
    return out
end

To check correctness. Note 
$$
\mathbf{M}^T \mathbf{1} = \mathbf{0}
$$
and
$$
\mathbf{M} \mathbf{x} = \mathbf{0}
$$
for stationary distribution $\mathbf{x}$.

Download the solution file `pgrksol.csv.gz`. **Do not put this file in your Git**. You will lose points if you do. You can add a line `pgrksol.csv.gz` to your `.gitignore` file.

In [50]:
using CodecZlib, DelimitedFiles

isfile("pgrksol.csv.gz") || download("https://raw.githubusercontent.com/ucla-biostat-257/2022spring/master/hw/hw4/pgrksol.csv.gz", pwd())
xsol = open("pgrksol.csv.gz", "r") do io
    vec(readdlm(GzipDecompressorStream(io)))
end

916428-element Vector{Float64}:
 3.3783428216975054e-5
 2.0710155392568165e-6
 3.663065984832893e-6
 7.527510785028837e-7
 8.63328599674051e-7
 1.769418252415541e-6
 2.431230382883396e-7
 6.368417180141445e-7
 4.744973703681939e-7
 2.6895486110647536e-7
 3.18574314847409e-6
 7.375106374416742e-7
 2.431230382883396e-7
 ⋮
 1.1305006040148547e-6
 4.874825281822915e-6
 3.167946973112519e-6
 9.72688040308568e-7
 6.588614479285245e-7
 7.737011774300648e-7
 2.431230382883396e-7
 1.6219204214797293e-6
 3.912130060551738e-7
 2.431230382883396e-7
 7.296033831163157e-6
 6.330939996912478e-7

**You will lose all 35 points (Steps 1 and 2)** if the following statements throw AssertError.

In [51]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)

@assert norm(transpose(M) * ones(n)) < 1e-12

In [52]:
@assert norm(M * xsol) < 1e-12

### Step 2 (20 pts)

We want to benchmark the hot functions `mul!` to make sure they are efficient and allocate little memory.

In [53]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)
v, out = ones(n), zeros(n)
bm_mv = @benchmark mul!($out, $M, $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 400 samples with 1 evaluation.
 Range (min … max):  12.237 ms … 13.287 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     12.280 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.300 ms ± 86.972 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▆█▃▃▂                                                     
  ▃▄▇██████▇▇▇▅▄▄▃▃▃▂▁▃▃▃▁▁▂▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▂▁▂▁▁▂▂▂▂▁▂▁▁▂▁▁▁▂ ▃
  12.2 ms         Histogram: frequency by time        12.6 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [54]:
bm_mtv = @benchmark mul!($out, $(transpose(M)), $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 415 samples with 1 evaluation.
 Range (min … max):  11.797 ms …  13.149 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     11.821 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   11.844 ms ± 105.568 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂▆█▇▆▅▅▃▁▃ ▁                                                  
  █████████████▆▁▇▅▄▁▅▄▁▅▁▄▁▄▁▄▁▁▄▁▆▁▁▅▄▄▄▅▁▁▁▁▁▁▄▄▄▁▁▁▁▁▁▁▁▁▄ ▇
  11.8 ms       Histogram: log(frequency) by time      12.2 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [55]:
clamp(10 - median(bm_mv).memory / 100, 0, 10) + 
clamp(10 - median(bm_mtv).memory / 100, 0, 10)

20.0

**Hint**: My median run times are 30-40 ms and memory allocations are 0 bytes.

### Step 3 (20 pts)

Let's first try to solve the PageRank problem by the GMRES method for solving linear equations. 

In [56]:
using KrylovKit

# normalize in-degrees to be the start point
x0   = vec(sum(A, dims = 1)) .+ 1.0
x0 ./= sum(x0)

# right hand side
b = zeros(n)

# warm up (compilation)
linsolve(M, b, x0, issymmetric = false, isposdef = false, maxiter = 1) 
# output is complex eigenvalue/eigenvector
(x_gmres, info), time_gmres, = @timed linsolve(M, b, x0, issymmetric = false, isposdef = false)

(value = ([3.378342822237397e-5, 2.0710155392781265e-6, 3.663065985290286e-6, 7.527510785714565e-7, 8.633285997289003e-7, 1.769418252763817e-6, 2.431230382949412e-7, 6.368417180841316e-7, 4.7449737038222236e-7, 2.689548611137733e-7  …  3.167946974023204e-6, 9.726880410372258e-7, 6.588614478640726e-7, 7.737011774829105e-7, 2.431230382949412e-7, 1.6219204214483914e-6, 3.912130060690359e-7, 2.431230382949412e-7, 7.296033831434169e-6, 6.330939996774086e-7], ConvergenceInfo: one converged value after 3 iterations and 72 applications of the linear map;
norms of residuals are given by (7.821117064217091e-13,).
), time = 4.01407025, bytes = 1008307445, gctime = 0.018868958, gcstats = Base.GC_Diff(1008307445, 137, 0, 64151, 4, 0, 18868958, 2, 0))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [57]:
@assert norm(x_gmres - xsol) < 1e-8

GMRES should be reasonably fast. The points you'll get is

In [58]:
clamp(20 / time_gmres * 20, 0, 20)

20.0

**Hint**: My runtime is about 7-8 seconds.

### Step 4 (20 pts)

Let's first try to solve the PageRank problem by the Arnoldi method for solving eigen problems. 

In [59]:
# warm up (compilation)
eigsolve(M, x0, 1, :SR, issymmetric = false, maxiter = 1)
# output is complex eigenvalue/eigenvector
(vals, vecs, info), time_arnoldi, = @timed eigsolve(M, x0, 1, :SR, issymmetric = false)

(value = (ComplexF64[-1.6492313476466683e-13 + 0.0im], Vector{ComplexF64}[[0.005635826953806499 + 0.0im, 0.0003454914380782207 + 0.0im, 0.0006110808494125081 + 0.0im, 0.00012557561626038913 + 0.0im, 0.000144022405328065 + 0.0im, 0.0002951783050398912 + 0.0im, 4.055832828680787e-5 + 0.0im, 0.00010623935784854303 + 0.0im, 7.915671116200255e-5 + 0.0im, 4.486765066728047e-5 + 0.0im  …  0.0005284839899796178 + 0.0im, 0.00016226599147987382 + 0.0im, 0.0001099127383761334 + 0.0im, 0.00012907055855754612 + 0.0im, 4.055832828680787e-5 + 0.0im, 0.0002705723874320663 + 0.0im, 6.526302748336452e-5 + 0.0im, 4.055832828680787e-5 + 0.0im, 0.0012171406601332862 + 0.0im, 0.00010561415510704564 + 0.0im]], ConvergenceInfo: one converged value after 7 iterations and 99 applications of the linear map;
norms of residuals are given by (9.574727551255147e-14,).
), time = 7.428900208, bytes = 1580663984, gctime = 0.050564334, gcstats = Base.GC_Diff(1580663984, 213, 0, 66859, 20, 0, 50564334, 5, 0))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [60]:
@assert abs(Real(vals[1])) < 1e-8

In [61]:
x_arnoldi   = abs.(Real.(vecs[1]))
x_arnoldi ./= sum(x_arnoldi)
@assert norm(x_arnoldi - xsol) < 1e-8

Arnoldi should be reasonably fast. The points you'll get is

In [62]:
clamp(20 / time_arnoldi * 20, 0, 20)

20.0

**Hint**: My runtime is about 11-12 seconds.

## Q6 (5 pts) Results

List the top 20 pages you found and their corresponding PageRank score. Do they match the top 20 pages ranked according to in-degrees? 

#### Solution:

In [63]:
#Sorted PageRank scores
x_gmres;
r = sortperm(x_gmres);

pagerank = collect(zip(r, x_gmres[r]));
pagerank[(length(r) - 19):length(r)]

20-element Vector{Tuple{Int64, Float64}}:
 (819224, 0.0005765189745892721)
 (885606, 0.0005812660190093561)
 (213433, 0.0005894474426283155)
 (7315, 0.0005926642909356454)
 (173977, 0.000603115191151035)
 (908352, 0.000614622081513541)
 (425771, 0.0006168480317129871)
 (751385, 0.0006546558408633476)
 (765335, 0.0006762276023319394)
 (551830, 0.0006950751256780081)
 (558792, 0.0007021658710885689)
 (32164, 0.000705518268358324)
 (605857, 0.000710848395487534)
 (486981, 0.0007177642925494127)
 (504141, 0.0007575423487533373)
 (384667, 0.0007791031791058587)
 (537040, 0.0008899344805014019)
 (163076, 0.0008950559016092856)
 (41910, 0.0009120131809713043)
 (597622, 0.0009145812115938595)

Comparing this to the in-degree scores directly computed from $A$:

In [66]:
#Sorted in-degree scores
rank_indeg[(length(rank_indeg) - 19):length(rank_indeg)]

20-element Vector{Tuple{Int64, Int64}}:
 (687326, 3956)
 (173977, 3988)
 (384667, 4010)
 (765335, 4015)
 (213433, 4084)
 (407611, 4180)
 (459075, 4187)
 (558792, 4206)
 (41910, 4219)
 (551830, 4220)
 (828964, 4484)
 (605857, 4550)
 (819224, 4620)
 (163076, 4731)
 (885606, 4847)
 (32164, 5097)
 (751385, 5182)
 (504141, 5271)
 (597622, 5354)
 (537040, 6326)

The rankings are generally similar, but not identical.  Highly influential sites with a large number of in-degree links scored generally quite well in PageRank score.  For instance, the top PageRank site (597,622) ranks second by in-degree links.  The next four top PageRank sites scored 12th, 7th, 1st, and 18th respectively by in-degree.  Others, like site 486,981, scored highly in PageRank (7th), but didn't crack the in-degree top 20.  However, this aligns with what we should expect from the PageRank algorithm.  In addition to total in-degree count, PageRank also accounts for the relative importance of the linking sites.  A link from a highly influential site counts more than a link from a low-influence one.  So we can reason that sites whose PageRank score outpaces their in-degree ranking are linked to by more influential pages than average.  The PageRank algorithm provides a highly efficient means of accounting for these factors.

## Q7 Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on a network of one million nodes.

#### Solution:

Thanks!